<a href="https://colab.research.google.com/github/satani99/fastai_22_2/blob/main/foundations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import matplotlib.pyplot as plt
import random

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.Button(description='Click me')

In [4]:
w

Button(description='Click me', style=ButtonStyle())

In [5]:
def f(o): print('hi')


In [6]:
w.on_click(f)

In [7]:
from time import sleep

In [8]:
def slow_calculation():
  res = 0
  for i in range(5):
    res += i*i
    sleep(1)
  return res

In [9]:
slow_calculation()

30

In [10]:
def slow_calculation(cb=None):
  res = 0
  for i in range(5):
    res += i*i
    sleep(1)
    if cb: cb(i)
  return res

In [11]:
def show_progress(epoch): print(f"Awesome! We've finished epoch {epoch}!")

In [12]:
slow_calculation(show_progress)

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

In [13]:
slow_calculation(lambda o: print(f"Awesome! We've finished epoch {o}!"))

Awesome! We've finished epoch 0!
Awesome! We've finished epoch 1!
Awesome! We've finished epoch 2!
Awesome! We've finished epoch 3!
Awesome! We've finished epoch 4!


30

In [14]:
def show_progress(exclamation, epoch): print(f"{exclamation}! We've finished epoch {epoch}!")

In [15]:
slow_calculation(lambda o: show_progress("OK I guess", o))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [16]:
def make_show_progress(exclamation):
  def _inner(epoch): print(f"{exclamation}! We've finished epoch {epoch}!")
  return _inner

In [17]:
slow_calculation(make_show_progress("Nice!"))

Nice!! We've finished epoch 0!
Nice!! We've finished epoch 1!
Nice!! We've finished epoch 2!
Nice!! We've finished epoch 3!
Nice!! We've finished epoch 4!


30

In [18]:
from functools import partial

In [19]:
slow_calculation(partial(show_progress, "OK I guess"))

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [20]:
f2 = partial(show_progress, "OK I guess")

In [21]:
slow_calculation(f2)

OK I guess! We've finished epoch 0!
OK I guess! We've finished epoch 1!
OK I guess! We've finished epoch 2!
OK I guess! We've finished epoch 3!
OK I guess! We've finished epoch 4!


30

In [22]:
class ProgressShowingCallback():
  def __init__(self, exclamation="Awesome"): self.exclamation = exclamation
  def __call__(self, epoch): print(f"{self.exclamation}! We've finished epoch {epoch}!")

In [23]:
cb = ProgressShowingCallback("Just Super")

In [24]:
slow_calculation(cb)

Just Super! We've finished epoch 0!
Just Super! We've finished epoch 1!
Just Super! We've finished epoch 2!
Just Super! We've finished epoch 3!
Just Super! We've finished epoch 4!


30

In [25]:
def f(*a, **b): print(f"args: {a}; kwargs: {b}")

In [26]:
f(3, 'a', thing1='hello')

args: (3, 'a'); kwargs: {'thing1': 'hello'}


In [27]:
def g(a, b, c=0): print(a, b, c)

In [28]:
args = [1, 2]
kwargs = {'c': 3}
g(*args, **kwargs)

1 2 3


In [29]:
def slow_calculation(cb=None):
  res = 0
  for i in range(5):
    if cb: cb.before_calc(i)
    res += i*i
    sleep(1)
    if cb: cb.after_calc(i, val=res)
  return res

In [30]:
class PrintStepCallback():
  def before_calc(self, *args, **kwargs): print(f"About to start")
  def after_calc(self, *args, **kwargs): print(f"Done step")

In [31]:
slow_calculation(PrintStepCallback())

About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step
About to start
Done step


30

In [32]:
class PrintStatusCallback():
  def __init__(self): pass
  def before_calc(self, epoch, **kwargs): print(f"About to start: {epoch}")
  def after_calc(self, epoch, val, **kwargs): print(f"After {epoch}: {val}")

In [33]:
slow_calculation(PrintStatusCallback())

About to start: 0
After 0: 0
About to start: 1
After 1: 1
About to start: 2
After 2: 5
About to start: 3
After 3: 14
About to start: 4
After 4: 30


30

In [35]:
def slow_calculation(cb=None):
  res = 0
  for i in range(5):
    if cb and hasattr(cb, 'before_calc'): cb.before_calc(i)
    res += i*i
    sleep(1)
    if cb and hasattr(cb, 'after_calc'):
      if cb.after_calc(i, res):
        print("stopping early")
        break

  return res

In [39]:
class PrintAfterCallback():
  def before_calc(self, epoch):
    print(f"Calculating {epoch} epoch")
  def after_calc(self, epoch, val):
    print(f"After {epoch}: {val}")
    if val > 10: return True

In [40]:
slow_calculation(PrintAfterCallback())

Calculating 0 epoch
After 0: 0
Calculating 1 epoch
After 1: 1
Calculating 2 epoch
After 2: 5
Calculating 3 epoch
After 3: 14
stopping early


14

In [38]:
class SlowCalculator():
  def __init__(self, cb=None): self.cb, self.res = cb, 0

  def callback(self, cb_name, *args):
    if not self.cb: return
    cb = getattr(self.cb, cb_name, None)
    if cb: return cb(self, *args)

  def calc(self):
    for i in range(5):
      self.callback('before_calc', i)
      self.res += i*i
      sleep(1)
      if self.callback('after_calc', i):
        print("stopping early")
        break

In [42]:
class ModifyingCallback():
  def after_calc(self, calc, epoch):
    print(f"After {epoch}: {calc.res}")
    if calc.res > 10: return True
    if calc.res < 3: calc.res = calc.res * 2

In [43]:
calculator = SlowCalculator(ModifyingCallback())

In [45]:
calculator.calc()
calculator.res

After 0: 0
After 1: 1
After 2: 6
After 3: 15
stopping early


15

In [46]:
class SloppyAdder():
  def __init__(self, o): self.o = o
  def __add__(self, b): return SloppyAdder(self.o + b.o + 0.01)
  def __repr__(self): return str(self.o)

In [47]:
a = SloppyAdder(1)
b = SloppyAdder(2)
a+b

3.01

In [48]:
class A: a, b = 1, 2

In [49]:
a = A()

In [50]:
a.b

2

In [51]:
getattr(a, 'b')

2

In [57]:
getattr(a, 'b' if random.random()>0.5 else 'a')

1

In [58]:
class B:
  a, b = 1, 2
  def __getattr__(self, k):
    if k[0]=='_': raise AttributeError(k)
    return f"Hello from {k}"

In [59]:
b = B()

In [60]:
b.a

1

In [61]:
b.foo

'Hello from foo'

In [62]:
b.b

2